In [8]:
%run "./metrics/metrics"

In [205]:
import numpy as np

class Bootstrap():
    def __init__(self, X, y, n_samples, len_sample):
        self.n_samples = n_samples
        self.len_sample = len_sample
        self.X = np.array(X)
        self.y = np.array(y)
        self.sample_X = np.zeros((n_samples, len_sample, self.X.shape[1]))
        self.sample_y = np.zeros((n_samples, len_sample, self.y.shape[1]))
        for i in range(n_samples):
            idx = np.random.choice(np.arange(len(X)), len_sample, replace=True)
            self.sample_X[i] = self.X[idx]
            self.sample_y[i] = self.y[idx]
        #print('a', *self.sample_X)
        #print('b', *self.sample_y)
    
    def values(self):
        return self.sample_X, self.sample_y
        


In [39]:
X = [[1, 2], [3, 4], [5, 6]]
b = Bootstrap(X, X, 5, 3)


a [[ 3.  4.]
 [ 1.  2.]
 [ 3.  4.]] [[ 3.  4.]
 [ 1.  2.]
 [ 1.  2.]] [[ 3.  4.]
 [ 3.  4.]
 [ 3.  4.]] [[ 5.  6.]
 [ 1.  2.]
 [ 5.  6.]] [[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]]
b [[ 3.  4.]
 [ 1.  2.]
 [ 3.  4.]] [[ 3.  4.]
 [ 1.  2.]
 [ 1.  2.]] [[ 3.  4.]
 [ 3.  4.]
 [ 3.  4.]] [[ 5.  6.]
 [ 1.  2.]
 [ 5.  6.]] [[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]]


In [40]:
class Model():
    def __init__(self):
        pass
    def fit(self, X, y):
        pass
    def predict(self, X):
        pass
    def predict_params(self, X):
        pass
        

In [21]:
class Metric():
    def __init__(self, func=None):
        if func == None:
            self.function = lambda a, b: 0
        else:
            self.function = func
    def evaluate(self, model, X, y):
        return self.function(y, model.predict(X))
    

In [25]:
class Characteristic():
    def __init__(self):
        pass
    def evaluate(self, model):
        pass
    

In [26]:
class Comparison():
    def __init__(self):
        pass
    def evaluate(self, model, full_X, mask, y):
        pass

In [23]:
m = Metric()

In [24]:
rss = Metric(residual_square_sum)

In [142]:
class EvaluateInfo():
    def __init__(self, model, metrics = [], characteristics = [], comparisons = []):
        self.model = model
        self.characteristics = characteristics # characteristic of the fitted model itself
        self.metrics = metrics # compares y and y_pred
        self.comparisons = comparisons #compares y, y_pred, y_pred_with_reduced_features
        
    def fit(self, X_train, y_train, X_test, y_test, mask = None, n_samples=20, len_sample=None):
        if mask == None:
            mask = np.ones(X.shape[1], dtype=np.bool)
        self.mask = mask
        if len_sample is None:
            len_sample = X.shape[1]
        self.len_sample = len_sample
        self.n_samples = n_samples

        self.X_train = X_train
        self.y_train = y_train

        self.X_test = X_test
        self.y_test = y_test

In [206]:
from copy import deepcopy
class EvaluateStaticInfo(EvaluateInfo):
    #def __init__(self, model, metrics = [], characteristics = [], comparisons = []):
    #    super.__init__(self, model, metrics = [], characteristics = [], comparisons = [])
    def __eval(self):
        self.sample_X, self.sample_y = self.b.values()
        print(len(self.metrics), self.n_samples)
        result_metrics = np.zeros((len(self.metrics), self.n_samples))
        #result_characteristics = np.zeros(len(self.characteristics), n_samples)
        #result_comparisons = np.zeros(len(self.comparisons), n_samples)
        self.full = deepcopy(model.fit(self.X_train, self.y_train))
        
        model.fit((self.X_train.T[self.mask]).T, self.y_train)
        
        for it in range(self.n_samples):
            for (i, metric) in enumerate(self.metrics):
                #print("S", (self.sample_X[it].T[self.mask]).T, self.sample_y[it])
                result_metrics[i][it] = metric.evaluate(model, (self.sample_X[it].T[self.mask]).T, self.sample_y[it])
                
        return result_metrics

    
    def fit(self, X_train, y_train, X_test, y_test, mask = None, n_samples=20, len_sample=None):
        super(EvaluateStaticInfo, self).fit(X_train, y_train, X_test, y_test, mask, n_samples, len_sample)
        self.b = Bootstrap(self.X_test, self.y_test, self.n_samples, self.len_sample)
        return self.__eval()
        

In [207]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [208]:
import scipy.stats as sps

X = sps.norm.rvs(size=(10, 5))
X2 = sps.norm.rvs(size=(10, 5))
z = sps.uniform.rvs(size=(5, 3))
y = X @ z + sps.norm.rvs(scale=0.1, size=(10, 3))
y2 = X2 @ z + sps.norm.rvs(scale=0.1, size=(10, 3))


In [209]:
metric = Metric(residual_square_sum)


In [210]:
info = EvaluateStaticInfo(model=model, metrics = [metric])

In [211]:
info.fit(X, y, X2, y2)

1 20


array([[ 0.23110496,  0.1686415 ,  0.20663252,  0.1949358 ,  0.38208779,
         0.35858081,  0.1886878 ,  0.26887413,  0.2571931 ,  0.22150303,
         0.28648987,  0.18533387,  0.24660494,  0.25487182,  0.4206269 ,
         0.19239849,  0.27814709,  0.20759798,  0.23509797,  0.18914183]])